In [1]:
import numpy as np
import pandas as pd

## 10. 데이터 준비하기

### 10.1 계층적 색인

In [5]:
# 예지 시리즈 타입 만들기 
# index 문으로 계층적으로 만들수 있음 
data = pd.Series(np.random.randn(9), 
                index = [['a', 'a','a', 'b','b', 'c','c','d', 'd'],
                        [1,2,3,1,3,1,2,2,3]] )
data

a  1   -0.018165
   2    0.005895
   3   -0.147613
b  1    1.287761
   3   -0.521257
c  1    0.285939
   2    0.148899
d  2    0.204198
   3    2.021000
dtype: float64

In [29]:
# 인덱스를 확인해보면 멀티인덱스가 나옴 
data.index      

data['a']

data['b' : 'c']

data.iloc[-7: ]

a  3   -0.147613
b  1    1.287761
   3   -0.521257
c  1    0.285939
   2    0.148899
d  2    0.204198
   3    2.021000
dtype: float64

In [30]:
# unstack() 
# 지금 인덱스가 2개 포개져 쌓여있는데 
# 이걸 풀어주는 느낌임 
# 자료형은 dataframe 임 
data_stack = data.unstack() 
data_stack

# # 다시 스택하기 
data_stack.stack() 

# # reset_index()  
# # 보통 unstack() 은 reset_index() 랑 같이 씀 
data_stack.unstack().reset_index() 

,level_0,level_1,0
0,1,a,-0.018165
1,1,b,1.287761
2,1,c,0.285939
3,1,d,NaN
4,2,a,0.005895
5,2,b,NaN
6,2,c,0.148899
7,2,d,0.204198
8,3,a,-0.147613
9,3,b,-0.521257


### 10.2 데이터 합치기

- 데이터베이스 스타일로 DataFrame 합치기

- inner join

In [33]:
# 예제 df 
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

In [35]:
df1, df2 

(  key  data1
 0   b      0
 1   b      1
 2   a      2
 3   c      3
 4   a      4
 5   a      5
 6   b      6,
   key  data2
 0   a      0
 1   b      1
 2   d      2)

In [57]:
# inner join 
# merge() 함수 
# 컬럼이 같은 거만 join 
# 기본 inner join 임 ( how로 바꿀수 있음)
pd.merge(df1, df2, on = 'key' , how = 'inner')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [42]:
# right join 
pd.merge(df1, df2, on = 'key' , how='right' )

,key,data1,data2
0,a,2.0,0
1,a,4.0,0
2,a,5.0,0
3,b,0.0,1
4,b,1.0,1
5,b,6.0,1
6,d,NaN,2


In [43]:
# key 가 다르면 
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [48]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


- outer join

In [49]:
# df1 는 c 가 있음
# df2 는 d 가 있음
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

In [51]:
# outer 
# 외부 조인 
pd.merge(df1, df2, how= 'outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


- 색인 병합하기

In [52]:
# 간혹 컬럼과 인덱스를 상대로 join 할때 
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [53]:
# key 라는 컬럼에  a b c 이런게 있음 
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [54]:
# 얘는 인덱스에 a b c 이런게 있음 
right1

,group_val
a,3.5
b,7.0


In [55]:
# right_index : 오른족 테이블에서 는 index에서 가져와라 

pd.merge(left1, right1, left_on = 'key', right_index= True) 

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [58]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [59]:
# 인덱스로 join 해보기 
pd.merge(left2, right2, right_index= True, left_index = True)

,Ohio,Nevada,Missouri,Alabama
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [62]:
# join() 함수 
# 기본 index 기준으로 테이블을 join 함 
# left join 이 기본임 (당연히 how로 바꿀수 있음)
left2.join(right2) 

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [63]:
# join() 함수에서 컬럼 값 사용해보기 

left1.join(right1, on ='key', how= 'inner')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


- 축따라 이어붙이기

In [68]:
arr = np.arange(12).reshape(3,4)
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [69]:
# np.concatenate() 
# 기본 axis = 0 행축을 따름 
np.concatenate([arr, arr]) 


# 열 축 따라 만들기 
np.concatenate([arr,arr], axis= 1 )

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [72]:
# pd.concat() 
# 판다스에서 제공하는 이어붙이기 
# 기본 axis =0 
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [71]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [73]:
# axis = 1 
pd.concat([s1,s2,s3], axis =1 )

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [83]:
# 데이터 프레임 이어붙이기 
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [84]:
# df2 에 c 가 없어서 Nan 값 들어감 
pd.concat([df1, df2], ignore_index= True)

# igbnore_index = True 는 기존 인덱스 무시

,a,b,c,d
0,0.096014,-0.795181,-0.913616,0.673129
1,0.525844,-0.997172,1.448064,1.819910
2,-1.694665,-0.604915,-1.244661,1.026482
3,1.470147,0.409843,NaN,1.032939
4,-0.235381,-0.223456,NaN,-1.305400


In [85]:
# axis =1
# df2 가  2x3 이라서 마지막 행에 Nan 값들어감 
pd.concat([df1, df2], axis =1) 

,a,b,c,d,b,d,a
0,0.096014,-0.795181,-0.913616,0.673129,0.409843,1.032939,1.470147
1,0.525844,-0.997172,1.448064,1.819910,-0.223456,-1.305400,-0.235381
2,-1.694665,-0.604915,-1.244661,1.026482,NaN,NaN,NaN


### Workshop

- 축따라 이어붙이기

In [98]:
df1 = pd.DataFrame({ 'a': ['a0', 'a1', 'a2', 'a3'],
     
                   'b': ['b0', 'b1', 'b2', 'b3'],
                    'c': ['c0', 'c1', 'c2', 'c3']},
                    index=[0, 1, 2, 3])
 
df2 = pd.DataFrame({'a': ['a2', 'a3', 'a4', 'a5'],
                    'b': ['b2', 'b3', 'b4', 'b5'],
                    'c': ['c2', 'c3', 'c4', 'c5'],
                    'd': ['d2', 'd3', 'd4', 'd5']},
                    index=[2, 3, 4, 5])

In [99]:
df1

,a,b,c
0,a0,b0,c0
1,a1,b1,c1
2,a2,b2,c2
3,a3,b3,c3


In [100]:
df2

,a,b,c,d
2,a2,b2,c2,d2
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5


In [101]:
# 2개의 데이터프레임을 위, 아래 (행축으로) 이어붙이듯 연결하기
# 인덱스를 재 설정
pd.concat([df1, df2 ], ignore_index = True) 

,a,b,c,d
0,a0,b0,c0,NaN
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,a3,b3,c3,NaN
4,a2,b2,c2,d2
5,a3,b3,c3,d3
6,a4,b4,c4,d4
7,a5,b5,c5,d5


In [102]:
# 2개의 데이터프레임을 좌, 우 (열축으로) 이어붙이듯 연결하기
pd.concat([df1, df2], axis =1 , ignore_index = True)

,0,1,2,3,4,5,6
0,a0,b0,c0,NaN,NaN,NaN,NaN
1,a1,b1,c1,NaN,NaN,NaN,NaN
2,a2,b2,c2,a2,b2,c2,d2
3,a3,b3,c3,a3,b3,c3,d3
4,NaN,NaN,NaN,a4,b4,c4,d4
5,NaN,NaN,NaN,a5,b5,c5,d5


In [103]:
sr1 = pd.Series(['e0', 'e1', 'e2', 'e3'], name='e')
sr2 = pd.Series(['f0', 'f1', 'f2'], name='f', index=[3, 4, 5])
sr3 = pd.Series(['g0', 'g1', 'g2', 'g3'], name='g')

In [109]:
# df1과 sr1을 좌, 우(열축으로) 이어붙이듯 연결하기
pd.concat([df1, sr1], axis = 1 )

,a,b,c,e
0,a0,b0,c0,e0
1,a1,b1,c1,e1
2,a2,b2,c2,e2
3,a3,b3,c3,e3


- 데이터베이스 스타일로 DataFrame 합치기

In [111]:
df1 = pd.read_excel('./datasets/stock price.xlsx')
df2 = pd.read_excel('./datasets/stock valuation.xlsx')

In [112]:
df1

,id,stock_name,value,price
0,128940,한미약품,59385.666667,421000
1,130960,CJ E&M,58540.666667,98900
2,138250,엔에스쇼핑,14558.666667,13200
3,139480,이마트,239230.833333,254500
4,142280,녹십자엠에스,468.833333,10200
5,145990,삼양사,82750.000000,82000
6,185750,종근당,40293.666667,100500
7,192400,쿠쿠홀딩스,179204.666667,177500
8,199800,툴젠,-2514.333333,115400
9,204210,모두투어리츠,3093.333333,3475


In [113]:
df2

,id,name,eps,bps,per,pbr
0,130960,CJ E&M,6301.333333,54068,15.695091,1.829178
1,136480,하림,274.166667,3551,11.489362,0.887074
2,138040,메리츠금융지주,2122.333333,14894,6.313806,0.899691
3,139480,이마트,18268.166667,295780,13.931338,0.860437
4,145990,삼양사,5741.000000,108090,14.283226,0.758627
5,161390,한국타이어,5648.500000,51341,7.453306,0.820007
6,181710,NHN엔터테인먼트,2110.166667,78434,30.755864,0.827447
7,185750,종근당,3990.333333,40684,25.185866,2.470259
8,204210,모두투어리츠,85.166667,5335,40.802348,0.651359
9,207940,삼성바이오로직스,4644.166667,60099,89.790059,6.938551


In [114]:
# id 를 조인조건으로 해서 inner join 
pd.merge(df1, df2, on = 'id', how = 'inner')

,id,stock_name,value,price,name,eps,bps,per,pbr
0,130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068,15.695091,1.829178
1,139480,이마트,239230.833333,254500,이마트,18268.166667,295780,13.931338,0.860437
2,145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090,14.283226,0.758627
3,185750,종근당,40293.666667,100500,종근당,3990.333333,40684,25.185866,2.470259
4,204210,모두투어리츠,3093.333333,3475,모두투어리츠,85.166667,5335,40.802348,0.651359


In [115]:
# id 를 조인 조건으로 해서 outer join
pd.merge(df1, df2, on= 'id' , how= 'outer')

,id,stock_name,value,price,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000.0,NaN,NaN,NaN,NaN,NaN
1,130960,CJ E&M,58540.666667,98900.0,CJ E&M,6301.333333,54068.0,15.695091,1.829178
2,138250,엔에스쇼핑,14558.666667,13200.0,NaN,NaN,NaN,NaN,NaN
3,139480,이마트,239230.833333,254500.0,이마트,18268.166667,295780.0,13.931338,0.860437
4,142280,녹십자엠에스,468.833333,10200.0,NaN,NaN,NaN,NaN,NaN
5,145990,삼양사,82750.000000,82000.0,삼양사,5741.000000,108090.0,14.283226,0.758627
6,185750,종근당,40293.666667,100500.0,종근당,3990.333333,40684.0,25.185866,2.470259
7,192400,쿠쿠홀딩스,179204.666667,177500.0,NaN,NaN,NaN,NaN,NaN
8,199800,툴젠,-2514.333333,115400.0,NaN,NaN,NaN,NaN,NaN
9,204210,모두투어리츠,3093.333333,3475.0,모두투어리츠,85.166667,5335.0,40.802348,0.651359


In [116]:
# 왼쪽 데이터프레임(df1)에서는 stock_name, 
# 오른쪽 데이터프레임(df2)에서는 name을 조인조건으로 하되
# left join

pd.merge(df1, df2, left_on = 'stock_name', right_on = 'name', how= 'left')

,id_x,stock_name,value,price,id_y,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000,NaN,NaN,NaN,NaN,NaN,NaN
1,130960,CJ E&M,58540.666667,98900,130960.0,CJ E&M,6301.333333,54068.0,15.695091,1.829178
2,138250,엔에스쇼핑,14558.666667,13200,NaN,NaN,NaN,NaN,NaN,NaN
3,139480,이마트,239230.833333,254500,139480.0,이마트,18268.166667,295780.0,13.931338,0.860437
4,142280,녹십자엠에스,468.833333,10200,NaN,NaN,NaN,NaN,NaN,NaN
5,145990,삼양사,82750.000000,82000,145990.0,삼양사,5741.000000,108090.0,14.283226,0.758627
6,185750,종근당,40293.666667,100500,185750.0,종근당,3990.333333,40684.0,25.185866,2.470259
7,192400,쿠쿠홀딩스,179204.666667,177500,NaN,NaN,NaN,NaN,NaN,NaN
8,199800,툴젠,-2514.333333,115400,NaN,NaN,NaN,NaN,NaN,NaN
9,204210,모두투어리츠,3093.333333,3475,204210.0,모두투어리츠,85.166667,5335.0,40.802348,0.651359


In [117]:
# 왼쪽 데이터프레임(df1)에서는 stock_name, 
# 오른쪽 데이터프레임(df2)에서는 name을 조인조건으로 하되
# right join
pd.merge(df1, df2, left_on = 'stock_name', right_on= 'name', how = 'right')

,id_x,stock_name,value,price,id_y,name,eps,bps,per,pbr
0,130960.0,CJ E&M,58540.666667,98900.0,130960,CJ E&M,6301.333333,54068,15.695091,1.829178
1,NaN,NaN,NaN,NaN,136480,하림,274.166667,3551,11.489362,0.887074
2,NaN,NaN,NaN,NaN,138040,메리츠금융지주,2122.333333,14894,6.313806,0.899691
3,139480.0,이마트,239230.833333,254500.0,139480,이마트,18268.166667,295780,13.931338,0.860437
4,145990.0,삼양사,82750.000000,82000.0,145990,삼양사,5741.000000,108090,14.283226,0.758627
5,NaN,NaN,NaN,NaN,161390,한국타이어,5648.500000,51341,7.453306,0.820007
6,NaN,NaN,NaN,NaN,181710,NHN엔터테인먼트,2110.166667,78434,30.755864,0.827447
7,185750.0,종근당,40293.666667,100500.0,185750,종근당,3990.333333,40684,25.185866,2.470259
8,204210.0,모두투어리츠,3093.333333,3475.0,204210,모두투어리츠,85.166667,5335,40.802348,0.651359
9,NaN,NaN,NaN,NaN,207940,삼성바이오로직스,4644.166667,60099,89.790059,6.938551


- 색인 병합으로 데이터프레임 합치기

In [132]:
df1 = pd.read_excel('./datasets/stock price.xlsx', index_col = 'id')
df2 = pd.read_excel('./datasets/stock valuation.xlsx', index_col = 'id')

In [133]:
# 데이터 프레임 인덱스 기준으로 병합(왼쪽 프레임 기준 )
df1.join(df2) 

,stock_name,value,price,name,eps,bps,per,pbr
id,,,,,,,,
128940,한미약품,59385.666667,421000,NaN,NaN,NaN,NaN,NaN
130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068.0,15.695091,1.829178
138250,엔에스쇼핑,14558.666667,13200,NaN,NaN,NaN,NaN,NaN
139480,이마트,239230.833333,254500,이마트,18268.166667,295780.0,13.931338,0.860437
142280,녹십자엠에스,468.833333,10200,NaN,NaN,NaN,NaN,NaN
145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090.0,14.283226,0.758627
185750,종근당,40293.666667,100500,종근당,3990.333333,40684.0,25.185866,2.470259
192400,쿠쿠홀딩스,179204.666667,177500,NaN,NaN,NaN,NaN,NaN
199800,툴젠,-2514.333333,115400,NaN,NaN,NaN,NaN,NaN


In [134]:
# 데이터프레임 인덱스를 기준으로 병합 (공통된 인덱스만)
df1.join(df2, how = 'inner')

,stock_name,value,price,name,eps,bps,per,pbr
id,,,,,,,,
130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068,15.695091,1.829178
139480,이마트,239230.833333,254500,이마트,18268.166667,295780,13.931338,0.860437
145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090,14.283226,0.758627
185750,종근당,40293.666667,100500,종근당,3990.333333,40684,25.185866,2.470259
204210,모두투어리츠,3093.333333,3475,모두투어리츠,85.166667,5335,40.802348,0.651359
